# Amazon Basics 2: Simple Storage Service

## Usage Notes

Amazon Simple Storage Service (S3) is an object store service provided by Amazon which allows you to store your data in the cloud.

https://aws.amazon.com/s3/

It's tightly coupled with a lot of different Amazon technology, and so if it ever goes down, bad things happen across Amazon.

https://aws.amazon.com/message/41926/

By the end of this exercise, you will understand how to do several different things with Amazon S3.

* How to create a user with permissions to buckets (among other things)
* How to create a bucket
* How to upload a file to Amazon S3
* How to download a file from Amazon S3
* How to make your files accessible to the world
* How to make your files accessible to other websites

This notebook will serve as a gentle introduction to the Amazon Console and Amazon Command Line interface (`awscli`) by having you interact with both in order to store and retrieve objects from Amazon S3. So while other parts might use the `aws` function defined in the previous notebook, this part only uses magic commands.

## Notebook Imports

In [ ]:
from __future__ import print_function
from aws_base import *

## Create a User

Before we begin, we'll need to create a user and set the credentials via `aws configure`, which we'd left blank in the previous notebook.

First, make sure you have an AWS account, and sign into the AWS Console.

* https://console.aws.amazon.com

You'll need to navigate to the **Identity and Access Management** service, also known as **IAM**. You can either type in IAM into the search bar, or you can click on the **Services** option in the top left and click on **IAM** afterwards.

From here, click on the **User** option, and click on the **Add User** button. Give the user a name, and choose the **Programmatic access** checkbox. On the permissions screen, select **Attach existing policies directly** and add ``AmazonS3FullAccess``.

When you click on the **Review** button, then the **Create User** button, your browser will auto-download a CSV file containing the user credentials. Then run the `aws configure` command and enter in your credentials.

<span style="color:crimson">**WARNING**: Do not share your AWS credentials or commit them in any files stored on Github. Essentially, whoever has access to these credentials will be able to do everything that you've given this user permission to do, which will steadily increase over the course of the tutorial, and it is not uncommon for people to end up with tens of thousands of dollars in charges for a misstep.</span>

Let's confirm that your information has been set by showing you that the `access_key` and `secret_key` have non-null values.

In [ ]:
!aws configure list

## Create a Bucket

Creating a bucket achieved through the `mb` command.

http://docs.aws.amazon.com/cli/latest/reference/s3/mb.html

The main thing you need to know about S3 buckets is that the name must be globally unique. This means that if you come up with a bucket name, it has to be one that no one else has ever used. For this reason, people will often add a prefix to it (the name of a GitHub repository, the name of a company, a Twitter handle, etc.) to reduce the risk of collisions.

Choose a name for your notebook, and then use `aws s3 mb BUCKET_NAME` in order to create your bucket.

In [ ]:
bucket_name = 'mdang.demo'

Now, the command! Inside of the shell commands, you can use curly braces `{}` to denote using a Python variable from the current Jupyter kernel. In this case, we'll reuse the `bucket_name` variable you specified above and pass it to the `mb` command.

In [ ]:
!aws s3 mb s3://{bucket_name} --region {region}

## Upload a File

Now that we have a bucket, we essentially have all the privileges we want on that bucket (unless you have funny S3 permissions, but I'm assuming you're using a personal account where no one subjected you to that). So now, the first thing we'll do is we'll upload a file.

First, a word of caution. **Amazon S3 is not free.** If you're still in a grace period known as Amazon Free Tier, some actions will be free (such as file transfers to and from S3), but other than that, you will be paying out of pocket for hosting files on S3. The price is fairly low (approximately $0.10 per gigabyte per month), but make sure you aren't caught off guard.

All that being said, let's first introduce you to the `%%writefile` magic command in Jupyter. This allows you to create a file with the specified name. If you already have subdirectories, you can also specify the subdirectory where you will store the file.

In [ ]:
%%writefile hello_world.svg
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">
    <a xlink:href="https://grow.liferay.com" target="_blank">
        <text x="0" y="32" font-size="32">Let It Grow</text>
    </a>
</svg>

Let's give it a look! You can use the `%%html` magic command in Jupyter to embed HTML straight into the notebook. Since we created an SVG in the above step, we can embed it.

In [ ]:
%%html
<embed src="hello_world.svg" height="32" />

Now that you have the file, in order to copy a file to a bucket, you have to use the `cp` command.

http://docs.aws.amazon.com/cli/latest/reference/s3/cp.html

The main thing to remember about files on Amazon S3 is that you're not actually working with a directory structure. Rather, you're working with a very flat key store.

http://docs.aws.amazon.com/AmazonS3/latest/dev/UsingObjects.html

However, by convention, a lot of things behave exactly as a directory would. For example, in using forward slashes to denote folder-like structures, you can simply add a trailing slash to the end of commands like `cp` and it's implicitly understood that you want to treat things like a folder.

In [ ]:
!aws s3 cp hello_world.svg s3://{bucket_name}/

## Download a File

Now imagine that your `hello_world.svg` became badly mangled because you were trying different things with it but ended up with something that was basically invisible.

In [ ]:
%%writefile hello_world.svg
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">
    <a xlink:href="https://grow.liferay.com" target="_blank">
        <text x="0" y="32" font-size="32" fill="#ffffff">Let It Grow</text>
    </a>
</svg>

And when you embedded it on a page, you were very sad, because you couldn't even see what you had written. It's almost like a malicious Jupyter notebook had replaced your `hello_world.svg` with one where the text was in invisible ink.

In [ ]:
%%html
<embed src="hello_world.svg" height="32" />

Luckily, the original is backed up on your S3 bucket.

In [ ]:
!aws s3 ls s3://{bucket_name}/

How do we get it back? It turns out that you do this with `cp`, just you switch the arguments around.

In [ ]:
!aws s3 cp s3://{bucket_name}/hello_world.svg .

And now we can re-embed our file and all is right with the world again.

In [ ]:
%%html
<embed src="hello_world.svg" height="32" />

## Access Control List

When copying files to Amazon S3, you can specify that other people (other than you, the person who own the bucket) may also be able to see this file. To understand what this means, let's get the theoretical public URL for the file we just uploaded and click on it when we're signed out of the web UI for Amazon Web Services.

In [ ]:
hello_world_url = 'http://s3-%s.amazonaws.com/%s/hello_world.svg' % (region, bucket_name)
print(hello_world_url)

Access denied. So, how do we fix this? To keep things simple, we're going to focus on recognizing that if you know when you're uploading the file that you want everyone to be able to see it, simply specify the `--acl` parameter when uploading your file, and everything will work out just fine. Let's go ahead and do that with our file.

In [ ]:
!aws s3 cp hello_world.svg s3://{bucket_name}/ --acl public-read

Now if you click on the link above, you'll find that your access is no longer denied.

## Content Headers

When making a file accessible to the public, you might find that large text files are bigger than you might like. For example, let's say you were to upload this Jupyter notebook to your bucket.

In [ ]:
!aws s3 cp basics2.ipynb s3://{bucket_name}/

Considering how little data is in this notebook, it's actually pretty big.

In [ ]:
!aws s3 ls s3://{bucket_name}/

It seems big for the web because you're likely used to gzipped file downloads, which aren't the default for Amazon S3 (after all, it's just a key-value store).

To solve that problem, when copying a file, you can also specify the content encoding that is always sent with the file. This means that you can gzip the content in advance and assume (usually correctly) that anyone accessing that file will have gzip enabled on their browser.

In [ ]:
for filename in ['basics2.ipynb', 'hello_world.svg']:
    !rm -f {filename}.gz
    !gzip -k {filename}
    !aws s3 cp {filename}.gz s3://{bucket_name}/{filename} --acl public-read --metadata-directive REPLACE

You'll now see that the file is much smaller in the case of our notebook (about 70% savings) slightly smaller in the case of our example SVG (about 25% savings). For files with high compression rates, you will likely find this step to be worthwhile.

In [ ]:
!aws s3 ls s3://{bucket_name}/

## Cross Origin Resource Sharing

Now that you know that you have a public URL, and now you know how to keep file sizes small for web browsers, what if you wanted to use those files for a website?

In general, simply using the URL directly works for things like images. However, since `amazonaws.com` is a separate domain from your actual website, you may run into issues where the browser is unable to view the file due to Cross-Origin Resource Sharing restrictions.

https://developer.mozilla.org/en-US/docs/Web/HTTP/Access_control_CORS

To overcome this, there is a command you can run which allows you to fine tune the way CORS works for your S3 buckets. For example, you might enable all GET requests so that anyone is able to download any public file located on your bucket from another domain. This is the `put-bucket-cors` command.

http://docs.aws.amazon.com/cli/latest/reference/s3api/put-bucket-cors.html

However, this command involves quotes, which are evaluated strangely when using bang inside of Jupyter notebook. Therefore, at this point, we switch to the `aws` function defined in the previous notebook.

In [ ]:
aws(
    's3api', 'put-bucket-cors', '--bucket', bucket_name,
    '--cors-configuration',
    '{"CORSRules":[{"AllowedOrigins":["*"],"AllowedMethods":["GET"]}]}')